In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("C:\\dev\\PythonHackLearning\\merged_patients.csv")

Q1 : Using prescriptive analysis, visualize patients options when Glucose is too high or too low. 

Solution in Progress

In [13]:
df['time'] = pd.to_datetime(df['time'])

df['glucose_diff'] = df.groupby('Patient_ID')['glucose'].diff()

In [9]:
patient_id = "HUPA0001P"
patient_df = df[df['Patient_ID'] == patient_id]

Q2 : Create a chart or graph to show, what is the ideal solution if heart_rate is low and glucose is rising ?

In [14]:
df['condition'] = (df['heart_rate'] < 60) & (df['glucose_diff'] > 0)

In [ ]:
plt.figure(figsize=(12,6))

sns.lineplot(x='time', y='glucose', data=patient_df, color='red', marker='o', label='Glucose')
sns.lineplot(x='time', y='heart_rate', data=patient_df, color='blue', marker='s', label='Heart Rate')

# Highlight prescriptive condition points
cond_points = patient_df[patient_df['condition']]
plt.scatter(cond_points['time'], cond_points['glucose'], 
            color='black', s=50, label='Low HR & Rising Glucose(Suggest: Light walk / adjust carb intake)')

plt.title(f"Prescriptive Chart for Patient {patient_id}: Low HR with Rising Glucose")
plt.xlabel("Time")
plt.ylabel("Glucose & HR Values")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Q3 : Using visualization, show patients with rising glucose at times during the day and suggest how this can be controlled

In [ ]:
df['rising_glucose'] = df['glucose_diff'] > 30

In [ ]:
day_glucose = (patient_df['time'].dt.hour >= 6) & (patient_df['time'].dt.hour <= 18)
patient_day_df = patient_df[day_glucose]
rising_glucose_df = patient_day_df[patient_day_df['rising_glucose']]

plt.figure(figsize=(12,6))

sns.lineplot(x='time', y='glucose', data=patient_day_df, marker='^', label='Glucose')

plt.scatter(rising_glucose_df['time'], rising_glucose_df['glucose'], 
            color='red', s=80, label='Rising Glucose')

plt.title(f"Patient {patient_id} Glucose Rising Periods (6 AM - 10 PM)")
plt.xlabel("Time")
plt.ylabel("Glucose (mg/dL)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
# Optional: create a summary table of interventions
interventions = rising_glucose_df[['time','glucose','carb_input']].copy()
interventions['recommendation'] = interventions['carb_input'].apply(
    lambda x: "Adjust insulin" if x > 0 else "Light activity"
)
print(interventions.head(10))


Q4 : Generate charts of patients whose glucose drops or rises within 30 mins and through prescriptive analysis recommend a solution.

Q5 : Use bar graph to show poor sleep quality afftects glucose level and recommend bedtime routine using prescriptive analysis

Q6 : Show relationship between poor sleep + high carb intake is equal to more post-meal glucose spikes. Show how carb intake can remidiate this.

Q7 : Suggest insulin intake for patients with high Glucose after meal based on gender.

Q8 : SHow corelation between Calories intake and glucose spike and recommend diet plan.

Q9: If a patient’s glucose keeps spiking after dinner despite taking insulin, what changes in their evening meal plan or insulin timing could help stabilize their levels? Can we visualize this with meal vs glucose curves?

Q10: When patients show both poor sleep quality and high resting heart rates, what lifestyle or medication adjustments can be suggested to improve recovery overnight? Can we plot sleep quality against heart rate to guide recommendations?